In [1]:
# Importing all the modules and packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('energydata_complete.csv')
df.head()

,recorded_timestamp,total_energy_use,lighting_energy_use,temp_sensor_1,humidity_sensor_1,temp_sensor_2,humidity_sensor_2,temp_sensor_3,humidity_sensor_3,temp_sensor_4,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,11-01-2016 17:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11-01-2016 17:10,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11-01-2016 17:20,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11-01-2016 17:30,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11-01-2016 17:40,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   recorded_timestamp    19735 non-null  object 
 1   total_energy_use      19735 non-null  int64  
 2   lighting_energy_use   19735 non-null  int64  
 3   temp_sensor_1         19735 non-null  float64
 4   humidity_sensor_1     19735 non-null  float64
 5   temp_sensor_2         19735 non-null  float64
 6   humidity_sensor_2     19735 non-null  float64
 7   temp_sensor_3         19735 non-null  float64
 8   humidity_sensor_3     19735 non-null  float64
 9   temp_sensor_4         19735 non-null  float64
 10  humidity_sensor_4     19735 non-null  float64
 11  temp_sensor_5         19735 non-null  float64
 12  humidity_sensor_5     19735 non-null  float64
 13  temp_sensor_6         19735 non-null  float64
 14  humidity_sensor_6     19735 non-null  float64
 15  temp_sensor_7      

In [4]:
df['recorded_timestamp'] = pd.to_datetime(df['recorded_timestamp'], format='%d-%m-%Y %H:%M')
df['recorded_timestamp'].head()

0   2016-01-11 17:00:00
1   2016-01-11 17:10:00
2   2016-01-11 17:20:00
3   2016-01-11 17:30:00
4   2016-01-11 17:40:00
Name: recorded_timestamp, dtype: datetime64[ns]

In [5]:
#Dropping the columns that are not needed
for i in range(29, 42):
    df.drop(['Unnamed: ' + str(i)], axis=1, inplace=True)
df.drop(['random_variable_1', 'random_variable_2'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   recorded_timestamp    19735 non-null  datetime64[ns]
 1   total_energy_use      19735 non-null  int64         
 2   lighting_energy_use   19735 non-null  int64         
 3   temp_sensor_1         19735 non-null  float64       
 4   humidity_sensor_1     19735 non-null  float64       
 5   temp_sensor_2         19735 non-null  float64       
 6   humidity_sensor_2     19735 non-null  float64       
 7   temp_sensor_3         19735 non-null  float64       
 8   humidity_sensor_3     19735 non-null  float64       
 9   temp_sensor_4         19735 non-null  float64       
 10  humidity_sensor_4     19735 non-null  float64       
 11  temp_sensor_5         19735 non-null  float64       
 12  humidity_sensor_5     19735 non-null  float64       
 13  temp_sensor_6   

In [6]:
# Checking for any missing values
df.isnull().sum()

recorded_timestamp      0
total_energy_use        0
lighting_energy_use     0
temp_sensor_1           0
humidity_sensor_1       0
temp_sensor_2           0
humidity_sensor_2       0
temp_sensor_3           0
humidity_sensor_3       0
temp_sensor_4           0
humidity_sensor_4       0
temp_sensor_5           0
humidity_sensor_5       0
temp_sensor_6           0
humidity_sensor_6       0
temp_sensor_7           0
humidity_sensor_7       0
temp_sensor_8           0
humidity_sensor_8       0
temp_sensor_9           0
humidity_sensor_9       0
external_temp           0
atmospheric_pressure    0
external_humidity       0
wind_speed_mps          0
visibility_km           0
dew_point_temp          0
dtype: int64

In [7]:
# Checking for outliers using IQR method
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    percent = (len(outliers) / len(df)) * 100
    print(f"{col}: {len(outliers)} outliers ({percent:.2f}%)")


total_energy_use: 2138 outliers (10.83%)
lighting_energy_use: 4483 outliers (22.72%)
temp_sensor_1: 515 outliers (2.61%)
humidity_sensor_1: 146 outliers (0.74%)
temp_sensor_2: 546 outliers (2.77%)
humidity_sensor_2: 235 outliers (1.19%)
temp_sensor_3: 217 outliers (1.10%)
humidity_sensor_3: 15 outliers (0.08%)
temp_sensor_4: 186 outliers (0.94%)
humidity_sensor_4: 0 outliers (0.00%)
temp_sensor_5: 179 outliers (0.91%)
humidity_sensor_5: 1330 outliers (6.74%)
temp_sensor_6: 515 outliers (2.61%)
humidity_sensor_6: 0 outliers (0.00%)
temp_sensor_7: 2 outliers (0.01%)
humidity_sensor_7: 42 outliers (0.21%)
temp_sensor_8: 71 outliers (0.36%)
humidity_sensor_8: 17 outliers (0.09%)
temp_sensor_9: 0 outliers (0.00%)
humidity_sensor_9: 21 outliers (0.11%)
external_temp: 440 outliers (2.23%)
atmospheric_pressure: 219 outliers (1.11%)
external_humidity: 239 outliers (1.21%)
wind_speed_mps: 214 outliers (1.08%)
visibility_km: 2522 outliers (12.78%)
dew_point_temp: 11 outliers (0.06%)


In [8]:
# Handling Outliers by capping

# Define a function to cap outliers using IQR method
def cap_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower, upper)
    return df

# Apply capping to the selected columns
for col in numeric_cols:
    df = cap_outliers_iqr(df, col)

In [9]:
# Checking for outliers again after capping

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    percent = (len(outliers) / len(df)) * 100
    print(f"{col}: {len(outliers)} outliers ({percent:.2f}%)")


total_energy_use: 0 outliers (0.00%)
lighting_energy_use: 0 outliers (0.00%)
temp_sensor_1: 0 outliers (0.00%)
humidity_sensor_1: 0 outliers (0.00%)
temp_sensor_2: 0 outliers (0.00%)
humidity_sensor_2: 0 outliers (0.00%)
temp_sensor_3: 0 outliers (0.00%)
humidity_sensor_3: 0 outliers (0.00%)
temp_sensor_4: 0 outliers (0.00%)
humidity_sensor_4: 0 outliers (0.00%)
temp_sensor_5: 0 outliers (0.00%)
humidity_sensor_5: 0 outliers (0.00%)
temp_sensor_6: 0 outliers (0.00%)
humidity_sensor_6: 0 outliers (0.00%)
temp_sensor_7: 0 outliers (0.00%)
humidity_sensor_7: 0 outliers (0.00%)
temp_sensor_8: 0 outliers (0.00%)
humidity_sensor_8: 0 outliers (0.00%)
temp_sensor_9: 0 outliers (0.00%)
humidity_sensor_9: 0 outliers (0.00%)
external_temp: 0 outliers (0.00%)
atmospheric_pressure: 0 outliers (0.00%)
external_humidity: 0 outliers (0.00%)
wind_speed_mps: 0 outliers (0.00%)
visibility_km: 0 outliers (0.00%)
dew_point_temp: 0 outliers (0.00%)


Feature Scaling

In [10]:
# Initialize StandardScaler
scaler = StandardScaler()

# Create a copy of the DataFrame
df_scaled = df.copy()

# Fit and transform the numeric columns
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

# Assign the scaled DataFrame back to df
df = df_scaled

Feature Engineering

In [12]:
# Step 2: Convert 'recorded_timestamp' to datetime and extract features
df['hour'] = df['recorded_timestamp'].dt.hour
df['day_of_week'] = df['recorded_timestamp'].dt.dayofweek
df['month'] = df['recorded_timestamp'].dt.month

In [13]:
# Step 3: Create aggregate temperature and humidity features
temp_cols = [col for col in df.columns if 'temp_sensor' in col]
humidity_cols = [col for col in df.columns if 'humidity_sensor' in col]

In [14]:
df['avg_indoor_temp'] = df[temp_cols].mean(axis=1)
df['max_indoor_temp'] = df[temp_cols].max(axis=1)
df['min_indoor_temp'] = df[temp_cols].min(axis=1)
df['range_indoor_temp'] = df['max_indoor_temp'] - df['min_indoor_temp']

In [15]:
df['avg_indoor_humidity'] = df[humidity_cols].mean(axis=1)
df['max_indoor_humidity'] = df[humidity_cols].max(axis=1)
df['min_indoor_humidity'] = df[humidity_cols].min(axis=1)
df['range_indoor_humidity'] = df['max_indoor_humidity'] - df['min_indoor_humidity']

In [16]:
# Step 4: Interaction features
df['temp_diff_indoor_outdoor'] = df['avg_indoor_temp'] - df['external_temp']
df['humidity_diff_indoor_outdoor'] = df['avg_indoor_humidity'] - df['external_humidity']

In [17]:
# Step 5: Time-based trend features (rolling mean of total energy use)
df['rolling_energy_use_mean'] = df['total_energy_use'].rolling(window=3, min_periods=1).mean()
